### **Transfer Learning**
Using MobileNet_V2

Dataset URL: http://download.tensorflow.org/example_images/flower_photos.tgz

## **Importing Libraries**

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

from sklearn.model_selection import train_test_split

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## **Download Zip File of Dataset**

In [ ]:
URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"
zip_file = tf.keras.utils.get_file(
    origin=URL,
    fname="flower_photos.tgz",
    extract=True
)

In [ ]:
zip_file

In [ ]:
!ls /root/.keras/datasets/

In [ ]:
base_dir = os.path.join(os.path.dirname(zip_file), "flower_photos")
base_dir

## **List category names present in dataset**

In [ ]:
class_names = [dir for dir in os.listdir(base_dir) if os.path.isdir(base_dir+"/"+dir)]
class_names

In [ ]:
filenames = list(glob.glob(base_dir+"/*/*.jpg"))
filenames[0]

In [ ]:
print("Total no.of images present in Dataset: {}".format(len(filenames)))

In [ ]:
categories = []
for filename in filenames:
  category = filename.split("/")[-2]
  categories.append(category)

## **Making a Dataframe with filenames and Their categories**

In [ ]:
dataframe = pd.DataFrame({
    "filename": filenames,
    "category": categories
})

In [ ]:
dataframe.head()

In [ ]:
dataframe['category'].value_counts().plot.bar()

## **Plotting a random image**

In [ ]:
random_image = np.random.choice(filenames)
random_image = tf.keras.preprocessing.image.load_img(random_image)
random_image

In [ ]:
random_image = tf.keras.preprocessing.image.img_to_array(random_image)
random_image.shape

Note: All images are in various shapes

## **Train, test, val Splits**

In [ ]:
train_full_df, test_df = train_test_split(dataframe, test_size=0.1, random_state=40)
train_df, val_df = train_test_split(train_full_df, test_size=0.1, random_state=40)

In [ ]:
train_df.shape

In [ ]:
train_length = train_df.shape[0]
test_length = test_df.shape[0]
val_length = val_df.shape[0]

## **Data Augmentation**

Note: If you are using preprocess_input then no need to rescale the image

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    preprocessing_function = preprocess_input
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    base_dir+"/*/",
    x_col='filename',
    y_col='category',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size = 32
)

In [ ]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = preprocess_input
)

In [ ]:
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    base_dir+"/*/",
    x_col='filename',
    y_col='category',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size = 32
)

## **Model Building**

In [ ]:
mobilenet = MobileNetV2(input_shape=[224, 224, 3], weights='imagenet', include_top=False)

In [ ]:
for layer in mobilenet.layers:
  layer.trainable = False

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(mobilenet.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

In [ ]:
model = tf.keras.models.Model(inputs=mobilenet.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), #from_logits=false beacause in last predcition layer we specified activation='softmax'
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=5,
    steps_per_epoch = train_length//32,
    validation_data=val_generator,
    validation_steps=val_length//32,
    callbacks = tf.keras.callbacks.EarlyStopping(patience=3)
)

## **Plot Model Performace**

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
test_generator = val_datagen.flow_from_dataframe(
    test_df,
    base_dir+"/*/",
    x_col='filename',
    y_col='category',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size = 32
)

In [ ]:
model.evaluate(test_generator)

**Hello Guys! Welcome again**

In previous video we saw how to build a model using transfer learning.
Now we will look into how to predict a single image.

## **Visualization of Predicted Results**

1. First we need to know what are the indices assigned to labels by our Generators.

In [ ]:
class_indices = train_generator.class_indices
class_indices #A dictionary

In [ ]:
#Exact all keys from above dictionary
class_indices = list(class_indices.keys())
class_indices

In [ ]:
test_df.head() #Our test dataframe

In [ ]:
#Extract a random value from test_df
test_df.iloc[1, :].values #Gives row 1 values

In [ ]:
#Generate Random values
test_df.iloc[np.random.randint(0, test_length-1), :].values

In [ ]:
data = test_df.iloc[np.random.randint(0, test_length-1), :].values
image_path = data[0]
actual_label = data[1]

In [ ]:
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image_array = tf.keras.preprocessing.image.img_to_array(image)/255
plt.imshow(image_array)
plt.title(actual_label)
plt.show()

In [ ]:
#Gather all code at one place
data = test_df.iloc[np.random.randint(0, test_length-1), :].values
image_path = data[0]
actual_label = data[1]

image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image_array = tf.keras.preprocessing.image.img_to_array(image)/255
plt.imshow(image_array)
plt.title(actual_label)
plt.show()

Now It is generating random image at each iteration

**Points to Remember:**
1. When we are using single images, image should be resized into which model is actually trained. Here our model is trained on image size (224, 224)
2. We trained our model in batches. So we need to make this single image into single batch

In [ ]:
data = test_df.iloc[np.random.randint(0, test_length-1), :].values
image_path = data[0]
actual_label = data[1]

image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image_array = tf.keras.preprocessing.image.img_to_array(image)/255
image_batch = np.expand_dims(image_array, axis=0) #Batching our single image
model.predict(image_batch)

Model will produce 5 different probabilities for each image.Beacuse we have 5 classess. The max probability index is our predicted class

In [ ]:
data = test_df.iloc[np.random.randint(0, test_length-1), :].values
image_path = data[0]
actual_label = data[1]

image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image_array = tf.keras.preprocessing.image.img_to_array(image)/255
image_batch = np.expand_dims(image_array, axis=0) #Batching our single image
print("Predicted label: ",np.argmax(model.predict(image_batch)))

In [ ]:
data = test_df.iloc[np.random.randint(0, test_length-1), :].values
image_path = data[0]
actual_label = data[1]

image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image_array = tf.keras.preprocessing.image.img_to_array(image)/255
image_batch = np.expand_dims(image_array, axis=0) #Batching our single image
predicted_label = class_indices[np.argmax(model.predict(image_batch))]

print("Predicted label: {}, Actual Label: {}".format(predicted_label, actual_label))

Yeah Pretty Good But now we plot Images

In [ ]:
data = test_df.iloc[np.random.randint(0, test_length-1), :].values
image_path = data[0]
actual_label = data[1]

image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image_array = tf.keras.preprocessing.image.img_to_array(image)/255
image_batch = np.expand_dims(image_array, axis=0) #Batching our single image
predicted_label = class_indices[np.argmax(model.predict(image_batch))]
plt.imshow(image_array)
if actual_label == predicted_label:
  plt.title(predicted_label, color='green')
else:
  plt.title(predicted_label, color='red')

In [ ]:
plt.figure(figsize=(15,15))
for i in range(25):
  ax = plt.subplot(5,5,i+1)
  data = test_df.iloc[np.random.randint(0, test_length-1), :].values
  image_path = data[0]
  actual_label = data[1]

  image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
  image_array = tf.keras.preprocessing.image.img_to_array(image)/255
  image_batch = np.expand_dims(image_array, axis=0) #Batching our single image
  predicted_label = class_indices[np.argmax(model.predict(image_batch))]
  plt.imshow(image_array)
  if actual_label == predicted_label:
    plt.title(predicted_label, color='green')
  else:
    plt.title(predicted_label, color='red')

plt.show()

**WELCOME**